In [3]:
# -*- coding: utf-8 -*-
import os, re, csv, requests 
from datetime import datetime
from bs4 import BeautifulSoup 
from urllib.request import urlopen
import pandas as pd
import numpy as np

In [86]:
ymd = datetime.today().strftime("%y%m%d")
PATH = os.getcwd() # 원하는 폴더로 수정

## 1. 온통 청년 카드 정보 크롤링
filename = f"card_{ymd}.csv"
f = open(os.path.join(PATH, filename), "w", encoding="utf-8-sig", newline="")
writer = csv.writer(f)

columns_title = ["정책", "D-day", "기관명", "분야", "진행 여부"]
writer.writerow(columns_title)

f.close()

df = pd.read_csv(os.path.join(PATH, "card.csv"))

pol_names = []
ddays = []
org_names = []
cates = []
ongoings = []

for i in range(1, 44):
    url = f"https://www.youthcenter.go.kr/youngPlcyUnif/youngPlcyUnifList.do?plcyCmprInfo=&pageIndex={i}&plcyCmprInfo=&trgtJynEmp=&trgtJynEmp="

    a = requests.get(url)
    soup = BeautifulSoup(a.text, 'html.parser')
    for j in range(1, 13):
        answers = soup.select(f"#srchFrm > div.sch-result-wrap > div.result-list-wrap > ul > li:nth-child({j}) > div.result-card-box > div.tit-wrap")
        pol_name = ""
        for a in answers:
            pol_name += a.get_text().strip()
        pol_names.append(pol_name)

        answers = soup.select(f"#srchFrm > div.sch-result-wrap > div.result-list-wrap > ul > li:nth-child({j}) > div.result-card-box > div.organ-name > p.dday")
        dday = ""
        for a in answers:
            dday += a.get_text().strip().replace('D-day ', '').replace('일', '')
        ddays.append(dday)

        answers = soup.select(f"#srchFrm > div.sch-result-wrap > div.result-list-wrap > ul > li:nth-child({j}) > div.result-card-box > div.organ-name > p:nth-child(1)")
        
        org_name = ""
        for a in answers:
            org_name += a.get_text().strip()
        org_names.append(org_name)

        answers = soup.select(f"#srchFrm > div.sch-result-wrap > div.result-list-wrap > ul > li:nth-child({j}) > div.result-card-box > div.badge > span.cate")
        cate = ""
        for a in answers:
            cate += a.get_text().strip()
        cates.append(cate)
        
        answers = soup.select(f"#srchFrm > div.sch-result-wrap > div.result-list-wrap > ul > li:nth-child({j}) > div.result-card-box > div.badge > span.label.green")
        if not answers:
            answers = soup.select(f"#srchFrm > div.sch-result-wrap > div.result-list-wrap > ul > li:nth-child({j}) > div.result-card-box > div.badge > span.label.purple")
            if not answers:
                answers = soup.select(f"#srchFrm > div.sch-result-wrap > div.result-list-wrap > ul > li:nth-child({j}) > div.result-card-box > div.badge > span.label.red")
                if not answers:
                    answers = soup.select(f"#srchFrm > div.sch-result-wrap > div.result-list-wrap > ul > li:nth-child({j}) > div.result-card-box > div.badge > span.label.blue")

        ongoing = ""
        for a in answers:
            ongoing += a.get_text().strip()
        ongoings.append(ongoing)

print(pol_names)
print(ddays)
print(org_names)
print(cates)
print(ongoings)

df["정책"] = pol_names
df["D-day"] = ddays
df["D-day"] = pd.to_numeric(df["D-day"], errors='coerce').fillna(0).astype(int) # int로 설정해 둬야 추후 마감일 기준 D-day 계산이 가능함
df["기관명"] = org_names
df["분야"] = cates
df["진행 여부"] = ongoings



['청년 주택드림 청약통장', '일자리 채움 청년지원금(빈일자리 청년취업지원금)', '근로자 휴가비 지원사업', '국민취업지원제도', '중소기업 취업청년 전월세보증금 대출', '청년월세 특별지원', '청년 주거급여 지원', '청년도약계좌', '국민내일배움카드', '2024년 상반기  서울특별시 동부기술교육원 직업교육훈련생 모집', '2024년 구직지원금 지급', '청년 국가기술자격시험 응시료 지원사업', '경기도 청소년 자립두배통장', '서울시 청년수당', '청년 디딤돌 2배 적금', '청년 소상공인 창업응원금 지원', '청년사회서비스사업단(신체건강증진서비스)', '청년전용창업자금(창업기반 지원)', '서울미래인재 모집', '경기도 청소년 교통비 지원 사업', '주거안정월세대출', '청년전용 버팀목전세자금', '청년우대형 청약통장', '도시가스 절약 캐시백', '서울 안심소득 참여가구 모집', '중소기업 취업자 소득세 감면(청년)', '2024년 자립준비청년(청년 유형) 전세임대', '청년도전지원사업', '기후동행카드', '서울형 뉴딜 일자리', '청년월세 지원사업', '청년 주택임차보증금 이자 지원 사업', '청년창업기업 세금 면제', '안양덕현 청년임대주택(행복주택) 입주자 모집', '문화누리카드', '미래두배 청년통장', '구미시 청년 전월세보증금 대출이자 지원사업', '한국장학재단 창업기숙사', '중소기업취업청년 전월세보증금 대출', '청년희망적금 만기자 청년도약계좌 일시납입 지원', '대중교통비 환급 지원 사업(k-패스)', '부모급여 지원', '경상남도 청년 대중교통비 지원 사업', '청년문화예술패스', '청년전용 보증부 월세대출', '2024년 첫만남 이용권', '경기도 청소년 자립두배 통장 참가자 모집', '2024 여행가는 달 <대한민국 숙박세일 페스타>', '상반기 경기도 대학생 학자금 대출이자 지원', '청년공동체 활동지원사업(경기도)', '햇살론유스', '서울시 청년동행센터', '청년맞춤형 주택임차보증금 이자지원', '신진예술인 창작준비금

In [87]:
## 2. 마강일 계산하여 설정하기
# 현재 크롤링한 'D-day'와 '진행 여부'는 서비스 제공 시점에 따라 변동될 가능성이 있음.
# 기준점으로 마감일을 설정함.
import datetime

# today와 d_day가 주어진 상태라고 가정
today = datetime.date.today()

df["마감일"] = df["D-day"].apply(lambda x: today + datetime.timedelta(days=x) if x != 0 else np.nan)

df['정책'].replace('', np.nan, inplace=True)
df.dropna(subset=['정책'], inplace=True)

df.to_csv(f"card_{ymd}.csv")

In [9]:
## 3. 서비스 날짜 기준 '진행 여부' 할당
# (주의)'진행 여부'가 '진행 예정'인 경우 수동으로 사이트에서 '시작일', '마감일' 확인하여 추가한 뒤 "card_240312_ver2.csv" 파일을 만듦.

PATH = os.getcwd() # 원하는 폴더로 수정
df = pd.read_csv(os.path.join(PATH, "card_240312_ver2.csv"))

# '마감일', '시작일' 열을 datetime 객체로 변환
df['마감일'] = pd.to_datetime(df['마감일'])
df['시작일'] = pd.to_datetime(df['시작일'])

# 오늘 날짜 구하기
today = pd.to_datetime('today')

# '진행 여부_new' 열 생성을 위한 사용자 정의 함수
def check_status(row):
    if row['진행 여부'] == '상시':
        return '상시'
    elif row['진행 여부'] == '진행 예정':
        if row['시작일'] < today:
            if row['마감일'] < today:
                return '신청 마감'
            else:
                return '진행중'
        elif row['시작일'] >= today:
            return '진행 예정'
        else:
            return '사이트 확인'
    elif row['마감일'] < today:
        return '신청 마감'
    else:
        return '진행중'

# apply 함수를 사용하여 각 행에 대해 check_status 함수 적용
df['진행 여부_new'] = df.apply(check_status, axis=1)

df.head()

,Unnamed: 0,정책,D-day,기관명,분야,진행 여부,마감일,시작일,진행 여부_new
0,0,청년 주택드림 청약통장,294,국토교통부,주거분야,진행중,2024-12-31,NaT,진행중
1,1,일자리 채움 청년지원금(빈일자리 청년취업지원금),0,고용노동부,일자리분야,상시,NaT,NaT,상시
2,2,근로자 휴가비 지원사업,294,문화체육관광부,복지.문화분야,진행중,2024-12-31,NaT,진행중
3,3,국민취업지원제도,0,고용노동부,일자리분야,상시,NaT,NaT,상시
4,4,중소기업 취업청년 전월세보증금 대출,0,국토교통부,주거분야,상시,NaT,NaT,상시


In [10]:
## 4. 서비스 날짜 기준 'D-day' 할당
PATH = os.getcwd() # 원하는 폴더로 수정
df = pd.read_csv(os.path.join(PATH, "card_240312.csv"))

# '마감일' 열을 datetime 객체로 변환
df['마감일'] = pd.to_datetime(df['마감일'])

# 오늘 날짜 구하기
today = pd.to_datetime('today')

# d-day 계산: '마감일'에서 오늘 날짜를 빼서 남은 일수 계산
df['D-day_new'] = (df['마감일'] - today).dt.days + 1

df.head()

,Unnamed: 0,정책,D-day,기관명,분야,진행 여부,마감일,시작일,진행 여부_new
42,42,경상남도 청년 대중교통비 지원 사업,0,경남,복지.문화분야,진행 예정,NaT,NaT,사이트 확인
43,43,청년문화예술패스,0,문화체육관광부,복지.문화분야,진행 예정,2024-12-31,2024-03-29,진행 예정
112,112,인천형 천원의 아침밥 지원,0,인천,복지.문화분야,진행 예정,NaT,NaT,사이트 확인
299,299,청년내일저축계좌(광주시),0,경기 광주시,복지.문화분야,진행 예정,2024-05-31,2024-05-01,진행 예정
306,306,청년 신혼부부 주거자금 대출이자 지원,0,충북,주거분야,진행 예정,2024-12-31,2024-07-01,진행 예정
318,318,청년농업인 영농정착 지원사업,0,광주 북구,일자리분야,진행 예정,2024-12-31,2024-04-01,진행 예정
343,343,2024년 안동시 청년 어학·자격시험 응시료 지원사업,0,경북 안동시,교육분야,진행 예정,2024-06-30,2024-05-01,진행 예정
